## Begin to Automate Your  Workflow With Functions

Functions can be tricky to learn. In this lesson, you will see how to break down a workflow and create 
a new function that can be used over and over on your data. 

You will want to use functions for this automation assignment. 

In [1]:
# Import necessary packages
import os
from glob import glob
import matplotlib.pyplot as plt
import numpy as np
from shapely.geometry import box
import geopandas as gpd
import rioxarray as rxr
import xarray as xr
from rasterio.plot import plotting_extent
from shapely.geometry import mapping
import earthpy as et
import earthpy.spatial as es
import earthpy.plot as ep

# Get data and set working directory
data = et.data.get_data('cold-springs-fire')
os.chdir(os.path.join(et.io.HOME,
                      "earth-analytics",
                      "data"))

In [2]:
# Get a list of each directory
path = os.path.join("ndvi-automation", "sites")

# Get a list of both site directories (We will talk more about automation next week)
sites = glob(path + "/*/")
# Get the site name
site_name = os.path.basename(os.path.normpath(sites[0]))


In [3]:
# Open up the shapefile for clipping your landsat data to the study area
vector_dir = os.path.join(sites[0],
                          "vector")

# Open crop boundary
site_boundary_path = os.path.join(vector_dir,  site_name + "-crop.shp")
crop_bound = gpd.read_file(site_boundary_path)

In [4]:
landsat_dir = os.path.join(sites[0],
                           "landsat-crop")
# This is the crop folder containing all of the .tif files
landsat_dirs = glob(os.path.join(landsat_dir, "LC08*"))
landsat_dirs.sort()

In [14]:
# Select just a single directory and grab bands 4-5 from the directory
adir = landsat_dirs[3]
# Open bands
band_paths = glob(os.path.join(adir, "*band*[4-5].tif"))
band_paths.sort()
band_paths

['ndvi-automation/sites/SJER/landsat-crop/LC080420342017022401T1-SC20181023152103/LC08_L1TP_042034_20170224_20170301_01_T1_sr_band4.tif',
 'ndvi-automation/sites/SJER/landsat-crop/LC080420342017022401T1-SC20181023152103/LC08_L1TP_042034_20170224_20170301_01_T1_sr_band5.tif']

In [15]:
# Open up the cloud mask layer

# Cloud no data vals for Landsat 8 -
vals = [328, 392, 840, 904, 1350, 352, 368, 416,
        432, 480, 864, 880, 928, 944, 992, 480, 992]

# Get cloud mask layer
qa_r = glob(os.path.join(adir, "*qa*"))

# Clip the cloud mask layer
cl_mask = rxr.open_rasterio(qa_r[0], masked=True).squeeze()
cl_mask_crop = cl_mask.rio.clip(crop_bound.geometry.apply(mapping))

# View unique values in the data -note that not every landsat band will have clouds to mask
np.unique(cl_mask_crop.values)

array([322., 324.], dtype=float32)

In [21]:
# Open a single band using rioxarray & mask to valid range then apply cloud mask
band = rxr.open_rasterio(band_paths[0], masked=True).squeeze()
band_crop = band.rio.clip(crop_bound.geometry)

# Specify the valid range of values for landsat
valid_range = (0, 10000)

if valid_range:
    mask = ((band_crop < valid_range[0]) | (band_crop > valid_range[1]))
    band_crop = band_crop.where(~mask)

band_crop = band_crop.where(~cl_mask_crop.isin(vals))
band_crop

<xarray.DataArray (y: 168, x: 143)>
array([[267., 306., 443., ..., 500., 548., 468.],
       [346., 346., 494., ..., 579., 567., 503.],
       [422., 384., 456., ..., 615., 616., 529.],
       ...,
       [446., 558., 650., ..., 690., 626., 675.],
       [546., 652., 630., ..., 656., 685., 699.],
       [470., 512., 562., ..., 478., 650., 606.]], dtype=float32)
Coordinates:
  * y            (y) float64 4.112e+06 4.112e+06 ... 4.107e+06 4.107e+06
  * x            (x) float64 2.546e+05 2.546e+05 ... 2.588e+05 2.588e+05
    band         int64 1
    spatial_ref  int64 0
Attributes:
    scale_factor:  1.0
    add_offset:    0.0

In [ ]:
def open_clean_bands(band_path,
                     crop_bound,
                     valid_range=None,
                     a_mask=None,
                     vals=None):
    """Open and mask a single landsat band using a pixel_qa layer.

    Parameters
    -----------
    band_path : string
        A path to the array to be opened
    crop_bound : geopandas GeoDataFrame
        A geopandas dataframe to be used to crop the raster data using 
        rasterio mask().
    valid_range : tuple (optional)
        A tuple of min and max range of values for the data. 
        Default = None
    a_mask : xarray DataArray
        An xarray DataArray with values that have not yet been set to 1
    vals : list
        A list of values needed to create the cloud mask

    Returns
    -----------
    band_crop : xarray DataArray
        An xarray DataArray with values that should be masked set to 1 
        for True (Boolean)
    """
    

    return band_crop

## What's fastest!!??

Below you can see different versions of the same workflow timed for speed!
The most efficient approach seems to be clipping and masking the NDVI layer
rather than doing that for each band!

In [ ]:
%%timeit
# Open and clean a single band
band_4 = open_clean_bands(band_path=band_paths[0],
                          crop_bound=crop_bound,
                          # The range of valid values for landsat can include negative values
                          # for this week let's stick with 0-10000
                          valid_range=(0, 10000),
                          a_mask=cl_mask_crop,
                          vals=vals)

# Open and clean a single band
band_5 = open_clean_bands(band_path=band_paths[1],
                          crop_bound=crop_bound,
                          # The range of valid values for landsat can include negative values
                          # for this week let's stick with 0-10000
                          valid_range=(0, 10000),
                          a_mask=cl_mask_crop,
                          vals=vals)

#  Then  calculate NDVI
ndvi = es.normalized_diff(band_5.values.astype('f4'), 
                   band_4.values.astype('f4'))


In [ ]:
%%timeit
# Alternatively do this in a loop and create a list output - cleaner code - same  amount of time
all_bands = []
for aband in band_paths:
    cleaned_band = open_clean_bands(band_path=aband,
                          crop_bound=crop_bound,
                          # The range of valid values for landsat can include negative values
                          # for this week let's stick with 0-10000
                          valid_range=(0, 10000),
                          a_mask=cl_mask_crop,
                          vals=vals)
    all_bands.append(cleaned_band)


# Then calculate NDVI - note ndvi here is not an xarray oobject - i  don't love that. 
ndvi_2 = es.normalized_diff(all_bands[1].values.astype('f4'), 
                   all_bands[0].astype('f4'))


In [ ]:
%%timeit
# Open Data - note that t his is not accounting for funky landsat values >10000 or <0
band_4 = rxr.open_rasterio(band_paths[0], masked=True).squeeze()
band_5 = rxr.open_rasterio(band_paths[1], masked=True).squeeze()

# Calculate NDVI
ndvi = (band_5 - band_4) / (band_5 + band_4)

ndvi_crop = ndvi.rio.clip(crop_bound.geometry.apply(mapping))

ndvi_crop = ndvi_crop.where(~cl_mask_crop.isin(vals))